In [1]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from multiprocessing import Pool, cpu_count, Manager, Value
import tensorflow as tf
from datetime import datetime, timedelta, date
import json
import os
import ctypes
from functools import partial

class Trade:
    def __init__(self, nome_ticker, data_inizio, data_fine, importo_da_investire, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione) -> None:
        self.SIMBOLO = nome_ticker
        self.DATA_INIZIO = data_inizio
        self.DATA_FINE = data_fine
        self.BILANCIO_INIZIALE = importo_da_investire
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.model = None
        self.valutazione_modello = None
        self.posizioni = None
        self.bilancio = None
        self.esito = None
        
    def avvia_trading(self, verbose=0) -> None:
        self.log = "\033[42m" + self.SIMBOLO + "\033[0m" + "\n"
        ticker = yf.download(self.SIMBOLO, start=self.DATA_INIZIO, end=self.DATA_FINE, progress=False)
        ticker.index = ticker.index.date
        str = f"Calcolo indicatori ticker {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"
        ticker = fx.crea_indicatori(ticker)
        ticker = fx.imposta_target(ticker)
        ticker.dropna(axis=0, inplace=True)

        str = f"Definizione features e target {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
        str = f"Previsione {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
        y_pred = self.model.predict(X)
        self.valutazione_modello = self.model.evaluate(X, Y)

        X = X[:, -1, :]

        df_X = ticker.loc[ticker.index.intersection(idx)]
        y_pred = y_pred.reshape(-1,)
        df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
        self.dati = pd.concat([df_X, df_y_pred], axis=1)

        self.bilancio = self.BILANCIO_INIZIALE
        pos_aperta = False
        tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
        giorni_posizione = 0
        self.posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=self.dati.index)

        for idx, row in self.dati.iterrows():
            if self.bilancio <= 0:
                break
            if pos_aperta == False:
                if row["prob_True"] > self.PROBABILITA_PER_ACQUISTO and self.bilancio > 0:
                    #COMPRA
                    prezzo_acquisto = row["Open"]
                    n_azioni = self.bilancio // prezzo_acquisto
                    prezzo_tot = n_azioni * prezzo_acquisto  
                    self.bilancio -= prezzo_tot          
                    tp = prezzo_acquisto * (1 + self.TP)
                    sl = prezzo_acquisto * (1 - self.SL)
                    pos_aperta = True
                    giorni_posizione = 0
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
            else:
                if giorni_posizione == self.GIORNI_POS:
                    prezzo_tot_vendita = row['Close'] * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    if prezzo_tot_vendita > prezzo_tot:
                        esito = 'VINCITA'
                    else:
                        esito = 'PERDITA'
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                if row["High"] >= tp:
                    prezzo_tot_vendita = tp * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                elif row["Low"] <= sl:
                    prezzo_tot_vendita = sl * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                else:
                    giorni_posizione += 1

        if pos_aperta: 
            self.bilancio += prezzo_tot

        tot_vincite = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "Esito"].count()
        tot_perdite = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "Esito"].count()
        max_vincita_value = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "P_L"].max(skipna=True)
        max_perdita_value = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "P_L"].min(skipna=True)
        max_vincita = int(max_vincita_value if max_vincita_value == max_vincita_value else 0)  # max_vincita_value == max_vincita_value è un modo per controllare se non è NaN
        max_perdita = int(max_perdita_value if max_perdita_value == max_perdita_value else 0)
        tot = int(fx.pct_change(self.BILANCIO_INIZIALE, self.bilancio))
        self.esito = (
            f"\nBilancio {self.SIMBOLO} = {round(self.bilancio, 2)}" + "\n" +
            f'Vincite: {tot_vincite}, vincita max: {max_vincita} %' + "\n" + 
            f'Perdite: {tot_perdite}, perdita max: {max_perdita} %' + "\n" +
            f'Totale: {tot} %'
        )
        if verbose == 1: print(self.esito)
        self.log += self.esito + "\n"

def _scarica(nome_simbolo, data_inizio, data_fine):
    try:
        ticker = yf.download(nome_simbolo, start=data_inizio, end=data_fine, progress=False)
        ticker.index = ticker.index.date
        ticker = fx.crea_indicatori(ticker)
        ticker.dropna(axis=0, inplace=True)
        return nome_simbolo, ticker, ""
    except Exception as e:
        return nome_simbolo, None, str(e)

def _callback_result(result, totale_processati, tot_tickers, ts_data_min, ts_data_max):
    nome_simbolo, ticker, error = result
    if error == "":
        ticker.to_parquet(f'tickers/{nome_simbolo}.parquet')
        with ts_data_min.get_lock():
            try:
                d = ticker.index.min()
                dt = datetime(d.year, d.month, d.day, hour=0, minute=0, second=0).timestamp()
                if dt < ts_data_min.value:
                    ts_data_min.value = dt
            except Exception as e:
                print(f"Errore su {nome_simbolo}: {str(e)}")
                
        with ts_data_max.get_lock():
            try:
                d = ticker.index.max()
                dt = datetime(d.year, d.month, d.day, hour=0, minute=0, second=0).timestamp()
                if dt > ts_data_max.value:
                    ts_data_max.value = dt
            except Exception as e:
                print(f"Errore su {nome_simbolo}: {str(e)}")

    else:
        print(f"Errore per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}")

class Borsa:
    def __init__(self, n_simboli_contemporanei, data_inizio, data_fine, bilancio_iniziale, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione):
        self.N_SIMBOLI = n_simboli_contemporanei
        self.DATA_INIZIO_STRINGA = data_inizio
        self.DATA_INIZIO = datetime.strptime(self.DATA_INIZIO_STRINGA, '%Y-%m-%d').date()
        self.DATA_FINE_STRINGA = data_fine
        self.DATA_FINE = datetime.strptime(self.DATA_FINE_STRINGA, '%Y-%m-%d').date()
        self.BILANCIO_INIZIALE = bilancio_iniziale
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': self.f1_score})
        self.posizioni = []
        self.bilancio = self.BILANCIO_INIZIALE
        self.lista_tickers = pd.read_parquet("lista_ticker.parquet")

        if os.path.exists(f'tickers/_indice.json'):
            with open('tickers/_indice.json', 'r') as jsonfile:
                indice = json.load(jsonfile)
            prima_data_str = indice['prima_data']
            ultima_data_str = indice['ultima_data']
            prima_data = datetime.strptime(prima_data_str, '%Y-%m-%d').date()
            ultima_data = datetime.strptime(ultima_data_str, '%Y-%m-%d').date()
            if (prima_data > self.DATA_INIZIO) or (ultima_data < self.DATA_FINE):
                self.scarica_tickers(min(prima_data, self.DATA_INIZIO), max(ultima_data, self.DATA_FINE))
        else:
            self.scarica_tickers(self.DATA_INIZIO, self.DATA_FINE)

    def scarica_tickers(self, data_inizio, data_fine) -> None:          
        tot_tickers = len(self.lista_tickers)
        totale_processati = Value('i', 0)
        data_min = datetime(1970, 1, 1)
        ts_data_min = Value(ctypes.c_double, data_min.timestamp())
        data_max = datetime.now()
        ts_data_max = Value(ctypes.c_double, data_max.timestamp())
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_callback_result, totale_processati=totale_processati, tot_tickers=tot_tickers, ts_data_min=ts_data_min, ts_data_max=ts_data_max)
            for i in range(0, tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, data_inizio, data_fine)
                p.apply_async(_scarica, args=param, callback=callback_with_args)

            p.close()
            p.join()
        data_min = datetime.fromtimestamp(ts_data_min.value).date()        
        data_max = datetime.fromtimestamp(ts_data_max.value).date()
        prima_data_str = data_min.strftime('%Y-%m-%d')
        ultima_data_str = data_max.strftime('%Y-%m-%d')
        indice = {
            'prima_data': prima_data_str,
            'ultima_data': ultima_data_str
        }
        with open('tickers/_indice.json', 'w') as jsonfile:
            json.dump(indice, jsonfile, indent=4)        

    def screener(self, data) -> None:
        pass

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning:



TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning:

Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and

In [2]:
if __name__ == "__main__":
    TP = 1
    SL = 0.01
    PROBABILITA_PER_ACQUISTO = 0.5
    BILANCIO_INIZIALE = 1000
    DATA_INIZIO = '2005-01-01'
    DATA_FINE = '2023-11-30'
    GIORNI_MAX_POSIZIONE = 20
    N_SIMBOLI = 10
    borsa = Borsa(N_SIMBOLI, DATA_INIZIO, DATA_FINE, BILANCIO_INIZIALE, PROBABILITA_PER_ACQUISTO, SL, TP, GIORNI_MAX_POSIZIONE)


1/2581) Completato ticker ZIP
2/2581) Completato ticker CARR
3/2581) Completato ticker OOMA
4/2581) Completato ticker CHEF
5/2581) Completato ticker IRTC
6/2581) Completato ticker NGVT
7/2581) Completato ticker DV
8/2581) Completato ticker SCPL
9/2581) Completato ticker CE
10/2581) Completato ticker PPBI
11/2581) Completato ticker NABL
12/2581) Completato ticker YPF
13/2581) Completato ticker PII
14/2581) Completato ticker WU
15/2581) Completato ticker HQY
16/2581) Completato ticker PODD
17/2581) Completato ticker SCS
18/2581) Completato ticker DG
19/2581) Completato ticker BKE
20/2581) Completato ticker UVE
21/2581) Completato ticker KRG
22/2581) Completato ticker AVY
23/2581) Completato ticker IBP
24/2581) Completato ticker DRVN
25/2581) Completato ticker TSCO
26/2581) Completato ticker CERE
27/2581) Completato ticker AWR
28/2581) Completato ticker VRRM
29/2581) Completato ticker FTV
30/2581) Completato ticker UI
31/2581) Completato ticker UNIT
32/2581) Completato ticker FANG
33/2581


1 Failed download:
['MONC']: Exception('%ticker%: No timezone found, symbol may be delisted')


75/2581) Completato ticker PK
Errore per MONC: 'Index' object has no attribute 'date'
76/2581) Completato ticker MONC
77/2581) Completato ticker DE
78/2581) Completato ticker GPS
79/2581) Completato ticker CMG
80/2581) Completato ticker WDAY
81/2581) Completato ticker RGLD
82/2581) Completato ticker SPGI
83/2581) Completato ticker SCVL
84/2581) Completato ticker GGG
85/2581) Completato ticker PLAY
86/2581) Completato ticker HTLF
87/2581) Completato ticker GPN
88/2581) Completato ticker IBEX
89/2581) Completato ticker CM
90/2581) Completato ticker STRL
91/2581) Completato ticker NMIH
92/2581) Completato ticker GOOGL
93/2581) Completato ticker ELF
94/2581) Completato ticker TW
95/2581) Completato ticker MEI



1 Failed download:
['ELN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1970-01-01 -> 2023-12-31)')


96/2581) Completato ticker WASH
Errore per ELN: 'Index' object has no attribute 'date'
97/2581) Completato ticker ELN
98/2581) Completato ticker PLYM
99/2581) Completato ticker WTW
100/2581) Completato ticker AIT
101/2581) Completato ticker HTZ
102/2581) Completato ticker RMD
103/2581) Completato ticker EC
104/2581) Completato ticker REY


...

2453/2581) Completato ticker PNW
2454/2581) Completato ticker QSR
2455/2581) Completato ticker AMCR
2456/2581) Completato ticker NXGN
Errore per ILTY: 'Index' object has no attribute 'date'
2457/2581) Completato ticker ILTY
2458/2581) Completato ticker CCBG
2459/2581) Completato ticker AOS
2460/2581) Completato ticker FE
2461/2581) Completato ticker DAKT
2462/2581) Completato ticker SNN
2463/2581) Completato ticker EXEL
2464/2581) Completato ticker DAR
2465/2581) Completato ticker DT
2466/2581) Completato ticker COTY
2467/2581) Completato ticker VSEC
2468/2581) Completato ticker LBRDA
2469/2581) Completato ticker DNOW
2470/2581) Completato ticker DXLG
2471/2581) Completato ticker CNOB
2472/2581) Completato ticker BK
2473/2581) Completato ticker SSTI
2474/2581) Completato ticker NE
2475/2581) Completato ticker NAVI
2476/2581) Completato ticker MDT
2477/2581) Completato ticker DORM
2478/2581) Completato ticker RPTX
2479/2581) Completato ticker FIVE
2480/2581) Completato ticker DAN
2481/2


1 Failed download:
['ATCO^PRH']: Exception('%ticker%: No timezone found, symbol may be delisted')


2484/2581) Completato ticker UCTT
2485/2581) Completato ticker FMAO
Errore per ATCO^PRH: 'Index' object has no attribute 'date'
2486/2581) Completato ticker ATCO^PRH
2487/2581) Completato ticker DCI
2488/2581) Completato ticker RGR
2489/2581) Completato ticker BEPC



1 Failed download:
['LDO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1970-01-01 -> 2023-12-31)')


2490/2581) Completato ticker FBP
2491/2581) Completato ticker WIT
Errore per LDO: 'Index' object has no attribute 'date'
2492/2581) Completato ticker LDO
2493/2581) Completato ticker ATRI
2494/2581) Completato ticker BDX
2495/2581) Completato ticker ENAV
2496/2581) Completato ticker MBIN
2497/2581) Completato ticker INN
2498/2581) Completato ticker TPX
2499/2581) Completato ticker GTES
2500/2581) Completato ticker CASY
2501/2581) Completato ticker BN
2502/2581) Completato ticker FTDR
2503/2581) Completato ticker EBF
2504/2581) Completato ticker SNDR
2505/2581) Completato ticker TARO
2506/2581) Completato ticker ED
2507/2581) Completato ticker ZIM
2508/2581) Completato ticker AMAL
2509/2581) Completato ticker CTRA
2510/2581) Completato ticker RDN
2511/2581) Completato ticker OLLI
2512/2581) Completato ticker DGX
2513/2581) Completato ticker BIPC
2514/2581) Completato ticker HOLX
2515/2581) Completato ticker THG
2516/2581) Completato ticker CLS
2517/2581) Completato ticker EVRI
Errore pe